In [1]:
import geopandas as gpd
import pandas as pd
import xarray as xr
import xvec
from shapely import Point

- Highways Data - Shape File from https://geodata.bts.gov/datasets/usdot::north-american-roads/about
- North Spokane Corrdor - data pulled from openstreetmap in osm_highways.py
- Spokane River Data - https://geo.wa.gov/search?q=river
- Historic Wildfires - https://www.mtbs.gov/

In [2]:
SPOKANE = {"geometry": [Point(-117.41, 47.65)]}
WATER_TREATMENT_FACILITIES = {"geometry": [Point(-117.357667, 47.659552), Point(-117.492292, 47.696336)], "ID": [0, 1]}
CRS = "EPSG:4326"
CONVERSION_CRS = "EPSG:2285"
METERS = 100 * 1609 # 100 miles in meters
gdf_spokane = gpd.GeoDataFrame(SPOKANE, crs=CRS)
gdf_spokane = gdf_spokane.to_crs(CONVERSION_CRS)
gdf_spokane["geometry"] = gdf_spokane.buffer(METERS)
gdf_spokane_50_mile_radius = gdf_spokane.to_crs(CRS)

historic_fires = gpd.read_file("/home/ubuntu/climate-risk-map/experimentation/student_projects/amazon_wildfire_risk_spring_2025/data/mtbs_perimeter_data/mtbs_perims_DD.shp")
historic_fires = historic_fires.to_crs(CRS)

gdf = gpd.read_file("/home/ubuntu/climate-risk-map/experimentation/student_projects/amazon_wildfire_risk_spring_2025/data/highways/NTAD_North_American_Roads_-6941702301048783378/North_American_Roads.shp")
gdf_wa = gdf.loc[(gdf["JURISNAME"]=="Washington")]
gdf_north_spokane_corridor = gpd.read_file("/home/ubuntu/climate-risk-map/experimentation/student_projects/amazon_wildfire_risk_spring_2025/data/highways/north_spokane_corridor.geojson")
gdf_rivers = gpd.read_file("/home/ubuntu/climate-risk-map/experimentation/student_projects/amazon_wildfire_risk_spring_2025/data/rivers.geojson")
gdf_spokane_river = gdf_rivers.loc[gdf_rivers["RIVER_NM"]=="Spokane River"][["OBJECTID", "RIVER_NM", "geometry"]].copy().rename({"OBJECTID": "ID"}, axis=1).set_index("ID")

gdf_water_treatment = gpd.GeoDataFrame(WATER_TREATMENT_FACILITIES, crs=CRS).set_index("ID")


line_gdfs = []

gdf_north_spokane_corridor["NAME"] = "north_spokane_corridor"
gdf_north_spokane_corridor["ID"] = gdf_north_spokane_corridor["id"]
line_gdfs.append(gdf_north_spokane_corridor[["ID", "NAME", "geometry"]])

highway_nums = ["I90", "U395", "U2"]
for highway in highway_nums:

    gdf_temp = gdf_wa.loc[gdf_wa["ROADNUM"]==highway].copy()
    gdf_temp["NAME"] = highway
    line_gdfs.append(gdf_temp[["ID", "NAME", "geometry"]])

gdf_highways = pd.concat(line_gdfs)
gdf_highways_spokane = gpd.sjoin(left_df=gdf_highways, right_df=gdf_spokane_50_mile_radius, how="inner")
gdf_highways_spokane = gdf_highways_spokane.drop(columns=["index_right"])
gdf_highways_spokane = gdf_highways_spokane.set_index("ID")

Skipping field node_ids: unsupported OGR type: 13


In [3]:
def zonal_aggregation_linestring(
    climate: xr.Dataset,
    climate_variable: str,
    infra: gpd.GeoDataFrame,
    x_dim: str,
    y_dim: str,
    time_col: str,
    aggregation: str
) -> pd.DataFrame:
    """Linestring cannot be zonally aggreated, so must be broken into points"""

    sampled_points = []
    for idx, row in infra.iterrows():
        line = row["geometry"]  # type == shapely.LineString
        points = list(line.coords)
        point_rows = [(idx, Point(point)) for point in points]
        sampled_points.extend(point_rows)

    if sampled_points:
        df_sampled_points = pd.DataFrame(
            sampled_points, columns=["ID", "geometry"]
        )
        gdf_sampled_points = gpd.GeoDataFrame(
            df_sampled_points, geometry="geometry", crs=infra.crs
        ).set_index("ID")
        ds_linestring_points = climate.xvec.extract_points(
            gdf_sampled_points.geometry, x_coords=x_dim, y_coords=y_dim, index=True
        )

        if time_col:
            df_linestring = (
                ds_linestring_points.stack(id_dim=("geometry", "decade_month"))
                .to_dataframe()
                .reset_index(drop=True)[
                    ["ID", "decade_month", "geometry"] + list(ds_linestring_points.data_vars)
                ]
            )
            df_linestring["decade"] = df_linestring["decade_month"].apply(lambda x: int(x[0:4]))
            df_linestring["month"] = df_linestring["decade_month"].apply(lambda x: int(x[-2:]))
            df_linestring.drop(columns=["decade_month"], inplace=True)
            group_by_columns = ["ID", "decade", "month"]
        else:
            df_linestring = ds_linestring_points.to_dataframe().reset_index(drop=True)[["ID"] + list(ds_linestring_points.data_vars)]
            group_by_columns = ["ID"]

        # TODO: At this step, we are left with OSM ids broken out into individual points.
        # Depending on the resolution of the climate dataset, there will be different exposure measures
        # along the entire linestring. Different segments of the same line will have different climate exposures,
        # while the entire line is considered one entity. For the time being, I am taking some simple means, mins, and max
        # to move forward with development. In the future, the ideal solution is to break up each osm_id into multiple line segments.
        # We can determine the segments by seeing where the mean, median, etc... values change along the points. We can group these series of points into multiple linestrings,
        # and then store the linestring segments in the database with their individual exposure values. A single osm id may be comprised of between 1 and N line segments.
        # The downside to this is extra segment geometries will need to be stored, possibly in their own tables. This also creates the eventual output dataset to the user more complicated
        # as a single entity may now have multiple records of exposure, one for each line segment.

        df_linestring = (
            df_linestring.drop_duplicates()
            .groupby(group_by_columns)
            .agg(
                {
                    climate_variable: aggregation,
                }
            )
            .reset_index()
        )
    else:
        df_linestring = pd.DataFrame()
    return df_linestring

In [4]:
ds_burn_probability = xr.open_dataset("s3://uw-crl/climate-risk-map/backend/climate/usda/BP_CONUS.zarr")
ds_burn_probability = ds_burn_probability.assign_coords({"lon": (((ds_burn_probability["lon"] + 180) % 360) - 180)})
ds_burn_probability = ds_burn_probability.sortby("lon")
ds_burn_probability = ds_burn_probability.sortby("lat")
ds_burn_probability = ds_burn_probability.sel({"lat": slice(44, 50), "lon": slice(-122, -115)})


ds_fwi_future = xr.open_dataset("s3://uw-crl/climate-risk-map/backend/climate/NEX-GDDP-CMIP6/DECADE_MONTH_ENSEMBLE/ssp370/fwi_decade_month_ssp370.zarr")
ds_fwi_historical = xr.open_dataset("s3://uw-crl/climate-risk-map/backend/climate/NEX-GDDP-CMIP6/DECADE_MONTH_ENSEMBLE/historical/fwi_decade_month_historical.zarr")
ds_fwi = xr.concat([ds_fwi_historical, ds_fwi_future], dim='decade_month')
ds_fwi = ds_fwi.assign_coords({"lon": (((ds_fwi["lon"] + 180) % 360) - 180)})
ds_fwi = ds_fwi.sortby("lon")
ds_fwi = ds_fwi.sortby("lat")
ds_fwi = ds_fwi.sel({"lat": slice(44, 50), "lon": slice(-122, -115)})

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7d34290a8cd0>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7d34290b9670>, 6846.971498595)])']
connector: <aiohttp.connector.TCPConnector object at 0x7d34290a8910>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7d3428e06990>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7d3429f46bd0>, 6846.973653146)])']
connector: <aiohttp.connector.TCPConnector object at 0x7d3429e80770>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7d3428e05a90>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7d3429f47890>, 6846.969451225)])']
connector: <aiohttp.connector.TCPConnector object at 0x7d3428e056d0>


In [12]:
df_burn_probability_highways = zonal_aggregation_linestring(
    climate=ds_burn_probability,
    climate_variable="burn_probability",
    infra=gdf_highways_spokane,
    x_dim="lon",
    y_dim="lat",
    time_col=None,
    aggregation="max"
)
df_fwi_highways = zonal_aggregation_linestring(
    climate=ds_fwi,
    climate_variable="ensemble_mean",
    infra=gdf_highways_spokane,
    x_dim="lon",
    y_dim="lat",
    time_col="decade_month",
    aggregation="mean"
)
df_fwi_highways = df_fwi_highways.rename(columns={"ensemble_mean": "fire_weather_index"})

ds_burn_probability_spokane_river = ds_burn_probability.xvec.extract_points(gdf_spokane_river.geometry, x_coords="lon", y_coords="lat", index=True)
df_burn_probability_spokane_river = ds_burn_probability_spokane_river.to_dataframe().reset_index()[["ID", "geometry"] + list(ds_burn_probability_spokane_river.data_vars)]
ds_fwi_spokane_river = ds_fwi.xvec.extract_points(gdf_spokane_river.geometry, x_coords="lon", y_coords="lat", index=True)
df_fwi_spokane_river = ds_fwi_spokane_river.stack(id_dim=("geometry", "decade_month")).to_dataframe().reset_index(drop=True)[["ID", "decade_month", "geometry", "ensemble_mean"]]
df_fwi_spokane_river["decade"] = df_fwi_spokane_river["decade_month"].apply(lambda x: int(x[0:4]))
df_fwi_spokane_river["month"] = df_fwi_spokane_river["decade_month"].apply(lambda x: int(x[-2:]))
df_fwi_spokane_river.drop(columns=["decade_month"], inplace=True)
df_fwi_spokane_river = df_fwi_spokane_river.rename(columns={"ensemble_mean": "fire_weather_index"})

ds_burn_probability_water_treatment = ds_burn_probability.xvec.extract_points(gdf_water_treatment.geometry, x_coords="lon", y_coords="lat", index=True)
df_burn_probability_water_treatment = ds_burn_probability_water_treatment.to_dataframe().reset_index()[["ID", "geometry"] + list(ds_burn_probability_water_treatment.data_vars)]
ds_fwi_water_treatment = ds_fwi.xvec.extract_points(gdf_water_treatment.geometry, x_coords="lon", y_coords="lat", index=True)
df_fwi_water_treatment = ds_fwi_spokane_river.stack(id_dim=("geometry", "decade_month")).to_dataframe().reset_index(drop=True)[["ID", "decade_month", "geometry", "ensemble_mean"]]
df_fwi_water_treatment["decade"] = df_fwi_water_treatment["decade_month"].apply(lambda x: int(x[0:4]))
df_fwi_water_treatment["month"] = df_fwi_water_treatment["decade_month"].apply(lambda x: int(x[-2:]))
df_fwi_water_treatment.drop(columns=["decade_month"], inplace=True)
df_fwi_water_treatment = df_fwi_water_treatment.rename(columns={"ensemble_mean": "fire_weather_index"})

In [30]:
highways_nearest_wildfire = gpd.sjoin_nearest(left_df=gdf_highways_spokane, right_df=historic_fires[["Ig_Date", "Incid_Name", "BurnBndAc", "geometry", "Comment"]], how="inner").drop(columns="index_right")
highways_burn_probability = gdf_highways_spokane.merge(df_burn_probability_highways, on="ID", how="left")
highways_fwi = gdf_highways_spokane.merge(df_fwi_highways, on="ID", how="left")

spokane_river_nearest_wildfire = gpd.sjoin_nearest(left_df=gdf_spokane_river, right_df=historic_fires[["Ig_Date", "Incid_Name", "BurnBndAc", "geometry", "Comment"]], how="inner").drop(columns="index_right")
spokane_river_burn_probability = df_burn_probability_spokane_river.copy()
spokane_river_fwi = df_fwi_spokane_river.copy()

water_treatment_nearest_wildfire = gpd.sjoin_nearest(left_df=gdf_water_treatment, right_df=historic_fires[["Ig_Date", "Incid_Name", "BurnBndAc", "geometry", "Comment"]], how="inner").drop(columns="index_right")
water_treatment_burn_probability = df_burn_probability_water_treatment.copy()
water_treatment_fwi = df_fwi_water_treatment.copy()

/home/ubuntu/climate-risk-map/experimentation/student_projects/amazon_wildfire_risk_spring_2025/.venv/lib/python3.13/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
/home/ubuntu/climate-risk-map/experimentation/student_projects/amazon_wildfire_risk_spring_2025/.venv/lib/python3.13/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
/home/ubuntu/climate-risk-map/experimentation/student_projects/amazon_wildfire_risk_spring_2025/.venv/lib/python3.13/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()'

In [33]:
highways_nearest_wildfire.to_csv("data/final/highways_nearest_historical_wildfire.csv")
highways_burn_probability.to_csv("data/final/highways_burn_probability_500m.csv", index=False)
highways_fwi.to_csv("data/final/highways_fire_weather_index.csv", index=False)

spokane_river_nearest_wildfire.to_csv("data/final/spokane_river_nearest_historical_wildfire.csv")
spokane_river_burn_probability.to_csv("data/final/spokane_river_burn_probability_500m.csv", index=False)
spokane_river_fwi.to_csv("data/final/spokane_river_fire_weather_index.csv", index=False)

water_treatment_nearest_wildfire.to_csv("data/final/water_treatment_nearest_historical_wildfire.csv")
water_treatment_burn_probability.to_csv("data/final/water_treatment_burn_probability_500m.csv", index=False)
water_treatment_fwi.to_csv("data/final/water_treatment_fire_weather_index.csv", index=False)